# [Exploration 08] Movielens 영화 추천 

## 프로젝트 설명 
- 추천시스템 실습
- Movielens 데이터셋을 사용    
`
$ wget http://files.grouplens.org/datasets/movielens/ml-1m.zip
`
    - Movielens : 영화에 대한 평점을 매긴 데이터
    - 평점 데이터는 explicit데이터이지만, implicit데이터로 간주하고 테스트 진행 
    - 3점 미만은 비선호로 간주해 데이터셋에서 제외
- MF모델 사용

## 프로젝트 목표
-

## 01. load and preprocess data
- column name을 생성합니다.

In [100]:
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']      # column name이 없기 때문에 생성
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
print(ratings.shape)
ratings.head()

(1000209, 4)


,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


- 정수로 된 user_id, movie_id, 평점(rating), 재생시간(timestamp)을 확인    
- 총 데이터 100만여개 확인

**3점 이상의 데이터만 남깁니다.**
- f string 소수점 코드 :.2%

In [101]:
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


**파일이름(ratings)와 겹치지 않도록 컬럼 *rating*의 이름을 *count*로 바꿉니다.**

In [102]:
ratings.rename(columns={'rating':'count'}, inplace=True)
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

**메타 데이터('movies.dat')으로 영화 제목을 확인합니다.**

In [103]:
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding = 'ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [104]:
movies['movie_id'].nunique()

3883

**ratings 데이터로 학습한 후, 메타 데이터로 예측한 영화의 제목, 장르를 확인해 모델을 평가할 수 있을 것으로 보입니다.**

- id, title을 확인하는 함수 생성

In [105]:
def id_to_title(id):
    return movies['title'][movies['movie_id']==id].values[0]

In [106]:
id_to_title(2)

'Jumanji (1995)'

In [107]:
def title_to_id(title):
    return movies['movie_id'][movies['title']==title].values[0].item()

In [108]:
title_to_id('Jumanji (1995)')

2

## 02. 데이터 파헤치기 
- 더 세부적인 정보를 얻기 위해 다음을 출력해보겠습니다.
    - ratings에 있는 유니크한 영화 개수
    - rating에 있는 유니크한 사용자 수
    - 가장 인기 있는 영화 30개(인기순)  

In [109]:
ratings.head()

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [110]:
# 유니크한 사용자 수 
ratings['user_id'].nunique()

6039

In [111]:
# 유니크한 영화 개수 
ratings['movie_id'].nunique()

3628

In [112]:
# 인기 많은 영화
movies_count = ratings.groupby('movie_id')['user_id'].count()   # 영화별 grouping, 사용자 수를 카운트
movies_count.sort_values(ascending=False).head(30)              # 상위 30편

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

**id로 되어있는 영화들, 영화의 제목을 알아봅시다!**

In [113]:
movies_30 = movies_count.sort_values(ascending=False).head(30)
movies_30.keys()

Int64Index([2858,  260, 1196, 1210, 2028,  589,  593, 1198, 1270, 2571,  480,
            2762,  608,  110, 1580,  527, 1197, 2396, 1617,  318,  858, 1265,
            1097, 2997, 2716,  296,  356, 1240,    1,  457],
           dtype='int64', name='movie_id')

In [114]:
for i in movies_30.keys():
    print(id_to_title(i))

American Beauty (1999)
Star Wars: Episode IV - A New Hope (1977)
Star Wars: Episode V - The Empire Strikes Back (1980)
Star Wars: Episode VI - Return of the Jedi (1983)
Saving Private Ryan (1998)
Terminator 2: Judgment Day (1991)
Silence of the Lambs, The (1991)
Raiders of the Lost Ark (1981)
Back to the Future (1985)
Matrix, The (1999)
Jurassic Park (1993)
Sixth Sense, The (1999)
Fargo (1996)
Braveheart (1995)
Men in Black (1997)
Schindler's List (1993)
Princess Bride, The (1987)
Shakespeare in Love (1998)
L.A. Confidential (1997)
Shawshank Redemption, The (1994)
Godfather, The (1972)
Groundhog Day (1993)
E.T. the Extra-Terrestrial (1982)
Being John Malkovich (1999)
Ghostbusters (1984)
Pulp Fiction (1994)
Forrest Gump (1994)
Terminator, The (1984)
Toy Story (1995)
Fugitive, The (1993)


**영화와 유저에 대한 통계, 영화 재생 시간에 대한 통계를 내보겠습니다.**

In [115]:
# 유저별 몇 편의 영화를 봤는지에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

In [116]:
# 유저별 play횟수 중앙값에 대한 통계
user_median = ratings.groupby('user_id')['timestamp'].median()
user_median.describe()

count    6.039000e+03
mean     9.705019e+08
std      8.916703e+06
min      9.567058e+08
25%      9.650976e+08
50%      9.720067e+08
75%      9.748566e+08
max      1.045440e+09
Name: timestamp, dtype: float64

In [117]:
ratings.describe()

,user_id,movie_id,count,timestamp
count,836478.000000,836478.000000,836478.000000,8.364780e+05
mean,3033.120626,1849.099114,3.958293,9.721628e+08
std,1729.255651,1091.870094,0.762280,1.206216e+07
min,1.000000,1.000000,3.000000,9.567039e+08
25%,1531.000000,1029.000000,3.000000,9.652795e+08
50%,3080.000000,1747.000000,4.000000,9.728388e+08
75%,4485.000000,2763.000000,5.000000,9.752064e+08
max,6040.000000,3952.000000,5.000000,1.046455e+09


## 02. 모델 검증을 위한 데이터 추가
- 내가 선호하는 영화를 5가지 골라서 rating에 추가합니다.
    - 필요한 데이터 : user_id, movie_id, count, timestamp

In [128]:
# 내가 좋아하는 영화 5편 추가
my_fav = ['Alien (1979)' , 'Sudden Manhattan (1996)' ,'Fall (1997)' ,'Mad City (1997)' ,'Alien: Resurrection (1997)']

# my_favorite을 movie_id로 변환 
my_fav_id = [title_to_id(i) for i in my_fav]

# user_id를 'miinkang'으로 정하고, 좋아하는 영화이니 모두 5점을 부여하겠습니다. 
my_movies = pd.DataFrame({'user_id': ['miinkang']*5, 'movie_id': my_fav_id, 'count':[5]*5, 'timestamp':[972162800]*5})

if not ratings.isin({'user_id':['miinkang']})['user_id'].any():  # user_id에 동일 id가 있는지 확인 후 추가
    ratings = ratings.append(my_movies)                            

ratings.tail(10)       # append는 맨 뒤에 붙인다. 

,user_id,movie_id,count,timestamp
0,6041,1214,5,972162800
1,6041,1558,5,972162800
2,6041,1574,5,972162800
3,6041,1667,5,972162800
4,6041,1690,5,972162800
0,miinkang,1214,5,972162800
1,miinkang,1558,5,972162800
2,miinkang,1574,5,972162800
3,miinkang,1667,5,972162800
4,miinkang,1690,5,972162800


In [127]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
artist_to_idx = {v:k for k,v in enumerate(movie_unique)}

print(user_unique, movie_unique)

[   1    2    3 ... 6039 6040 6041] [1193  661  914 ... 2909 1360 1558]


In [98]:
user_to_idx['miinkang']

6039

- 위 방법으로 바꾸려고 했지만, id중복이 되기 때문에 빠진 값으로 바꿔준다. 

In [125]:
for i in range(1, 6040):
    if i in ratings['user_id'].values:
        continue
    else:
        print(i)

3598


In [126]:
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

user_id column indexing Fail!!


## 03. CSR matrix 만들기 

In [55]:
ratings

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
0,7777,1214,5,972162800
1,7777,1558,5,972162800
2,7777,1574,5,972162800
3,7777,1667,5,972162800


In [73]:
print(num_user, num_movie)
max(ratings['user_id'])

6041 3629


7777

In [119]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings['user_id'], ratings['movie_id'])), shape=(num_user, num_movie))
csr_data

ValueError: row index exceeds matrix dimensions

## 03. 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.
## 04. CSR matrix를 직접 만들어 봅시다.
## 05. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.
## 06. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.
## 07. 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.
## 08. 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

# 정리     
---        
1. 데이터 준비와 전처리
2. 분석해 봅시다.
- ratings에 있는 유니크한 영화 개수
- rating에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)  
3. 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.
4. CSR matrix를 직접 만들어 봅시다.
5. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.
6. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.
7. 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.
8. 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.


# 루브릭 평가
---
1. **CSR matrix가 정상적으로 만들어졌다.**   
    - 사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.


2. **MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.**  
    - 사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.
    
     
3. **비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.**  
    - MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.

    
# 회고
---
## 더 공부하고 싶은 부분


## 느낀 점